In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, BatchNormalization
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.metrics import classification_report
from keras import optimizers
import datetime
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)


traindata = pd.read_csv('KDDTrain+.csv', header=None)
testdata = pd.read_csv('KDDTest+.csv', header=None)


columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'class',
    'difficulty_level'
]
traindata.columns=columns
testdata.columns=columns

del traindata["difficulty_level"]
del testdata["difficulty_level"]
traindata.info()
testdata.info()
trainlen=traindata.shape[0]

Using TensorFlow backend.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 42 columns):
duration                       125973 non-null int64
protocol_type                  125973 non-null object
service                        125973 non-null object
flag                           125973 non-null object
src_bytes                      125973 non-null int64
dst_bytes                      125973 non-null int64
land                           125973 non-null int64
wrong_fragment                 125973 non-null int64
urgent                         125973 non-null int64
hot                            125973 non-null int64
num_failed_logins              125973 non-null int64
logged_in                      125973 non-null int64
num_compromised                125973 non-null int64
root_shell                     125973 non-null int64
su_attempted                   125973 non-null int64
num_root                       125973 non-null int64
num_file_creations             125973 

In [2]:
alldata=pd.concat([traindata, testdata], axis=0, join='outer',ignore_index=True)
print(alldata)
#print(alldata['protocol_type'])
class_names=alldata['class'].unique()
print(class_names)

mask = alldata['class'] != 'normal' 
alldata.loc[mask, 'class'] = 'attack'
alldata['class'].value_counts()

#alldata.head()

        duration protocol_type     service  flag  src_bytes  dst_bytes  land  \
0              0           tcp    ftp_data    SF        491          0     0   
1              0           udp       other    SF        146          0     0   
2              0           tcp     private    S0          0          0     0   
3              0           tcp        http    SF        232       8153     0   
4              0           tcp        http    SF        199        420     0   
5              0           tcp     private   REJ          0          0     0   
6              0           tcp     private    S0          0          0     0   
7              0           tcp     private    S0          0          0     0   
8              0           tcp  remote_job    S0          0          0     0   
9              0           tcp     private    S0          0          0     0   
10             0           tcp     private   REJ          0          0     0   
11             0           tcp     priva

['normal' 'neptune' 'warezclient' 'ipsweep' 'portsweep' 'teardrop' 'nmap'
 'satan' 'smurf' 'pod' 'back' 'guess_passwd' 'ftp_write' 'multihop'
 'rootkit' 'buffer_overflow' 'imap' 'warezmaster' 'phf' 'land'
 'loadmodule' 'spy' 'perl' 'saint' 'mscan' 'apache2' 'snmpgetattack'
 'processtable' 'httptunnel' 'ps' 'snmpguess' 'mailbomb' 'named'
 'sendmail' 'xterm' 'worm' 'xlock' 'xsnoop' 'sqlattack' 'udpstorm']


normal    77053
attack    71463
Name: class, dtype: int64

In [3]:

#traindata.info()
#traindata.describe()
protocol_type_dummy=pd.get_dummies(alldata['protocol_type'])
alldata=pd.concat([alldata,protocol_type_dummy],axis=1)
del alldata['protocol_type']


service_dummy=pd.get_dummies(alldata["service"])
alldata=pd.concat([alldata,service_dummy],axis=1)
del alldata["service"]

flag_dummy=pd.get_dummies(alldata["flag"])
alldata=pd.concat([alldata,flag_dummy],axis=1)
del alldata["flag"]
print(alldata.head())

#print(protocol_type_dummy)

encode_text_index(alldata, 'class')
# Break into X (predictors) & y (prediction)
x, y = to_xy(alldata,'class')

traindata=alldata.ix[0:trainlen-1,:]
testdata=alldata.ix[trainlen:alldata.shape[0],:]

   duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
0         0        491          0     0               0       0    0   
1         0        146          0     0               0       0    0   
2         0          0          0     0               0       0    0   
3         0        232       8153     0               0       0    0   
4         0        199        420     0               0       0    0   

   num_failed_logins  logged_in  num_compromised ...  REJ  RSTO  RSTOS0  RSTR  \
0                  0          0                0 ...    0     0       0     0   
1                  0          0                0 ...    0     0       0     0   
2                  0          0                0 ...    0     0       0     0   
3                  0          1                0 ...    0     0       0     0   
4                  0          1                0 ...    0     0       0     0   

   S0  S1  S2  S3  SF  SH  
0   0   0   0   0   1   0  
1   0   0   0   0   1   

C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [4]:
#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


print(X_train.shape)
print(X_test.shape)
print(X.columns)
print(T.columns)


X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Columns: 122 entries, duration to SH
dtypes: float64(15), int64(23), uint8(84)
memory usage: 46.6 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int32
dtypes: int32(1)
memory usage: 492.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Columns: 122 entries, duration to SH
dtypes: float64(15), int64(23), uint8(84)
memory usage: 8.3 MB
C info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 1 columns):
class    22543 non-null int32
dtypes: int32(1)
memory usage: 88.1 KB
(125973, 1, 122)
(22543, 1, 122)
Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       ...
       'REJ', 'RSTO', 'RSTOS0', 'RSTR', 'S0', 'S1', 'S2

# Testing Time_ Model created with  all 122 features

In [5]:
from keras.models import load_model
# Recreate the exact same model purely from the file

new_model = load_model('all122featuresModel.h5')
time1= datetime.datetime.now()
new_predictions = new_model.predict(X_test)
time2= datetime.datetime.now()
time=((time2-time1)/22543)
print(time)


0:00:00.000020


# Testing Time_ Model created with 8 features

In [6]:
alldata.head()
#col_list=['dst_host_diff_srv_rate','dst_host_serror_rate','serror_rate','dst_host_same_src_port_rate','S0','dst_host_srv_serror_rate','class']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot','dst_host_srv_serror_rate','is_guest_login','icmp','REJ','dst_host_serror_rate','dst_host_srv_diff_host_rate','tcp','urp_i','dst_host_rerror_rate','finger']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count']


#10 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','ecr_i','http','private']

#8 feature set
col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_same_srv_rate','ecr_i','http','private']

#6 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','ecr_i','http']

#4 feature set
#col_list=['class','src_bytes', 'dst_bytes','ecr_i','http']

#3 feature set
#col_list=['class','src_bytes', 'ecr_i','http']

#col_list=['class','ecr_i','http','src_bytes','hot','dst_bytes','dst_host_same_srv_rate']
alldata2=alldata[col_list]
traindata=alldata2.ix[0:trainlen-1,:]
testdata=alldata2.ix[trainlen:alldata2.shape[0],:]


#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))

X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 8 columns):
src_bytes                 125973 non-null int64
dst_bytes                 125973 non-null int64
hot                       125973 non-null int64
count                     125973 non-null int64
dst_host_same_srv_rate    125973 non-null float64
ecr_i                     125973 non-null uint8
http                      125973 non-null uint8
private                   125973 non-null uint8
dtypes: float64(1), int64(4), uint8(3)
memory usage: 5.2 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int32
dtypes: int32(1)
memory usage: 492.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 8 columns):
src_bytes                 22543 non-null int64
dst_bytes                 22543 non-null int64
hot                       22543 no

C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [7]:
from keras.models import load_model
# Recreate the exact same model purely from the file

new_model = load_model('8featureModel.h5')
time1= datetime.datetime.now()
new_predictions = new_model.predict(X_test)
time2= datetime.datetime.now()
time=((time2-time1)/22543)
print(time)

0:00:00.000019


# Testing Time_ Model created with 6 features


In [8]:
alldata.head()
#col_list=['dst_host_diff_srv_rate','dst_host_serror_rate','serror_rate','dst_host_same_src_port_rate','S0','dst_host_srv_serror_rate','class']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot','dst_host_srv_serror_rate','is_guest_login','icmp','REJ','dst_host_serror_rate','dst_host_srv_diff_host_rate','tcp','urp_i','dst_host_rerror_rate','finger']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count']


#10 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','ecr_i','http','private']

#8 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_same_srv_rate','ecr_i','http','private']

#6 feature set
col_list=['class','src_bytes', 'dst_bytes','hot','count','ecr_i','http']

#4 feature set
#col_list=['class','src_bytes', 'dst_bytes','ecr_i','http']

#3 feature set
#col_list=['class','src_bytes', 'ecr_i','http']

#col_list=['class','ecr_i','http','src_bytes','hot','dst_bytes','dst_host_same_srv_rate']
alldata2=alldata[col_list]
traindata=alldata2.ix[0:trainlen-1,:]
testdata=alldata2.ix[trainlen:alldata2.shape[0],:]


#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))

X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 6 columns):
src_bytes    125973 non-null int64
dst_bytes    125973 non-null int64
hot          125973 non-null int64
count        125973 non-null int64
ecr_i        125973 non-null uint8
http         125973 non-null uint8
dtypes: int64(4), uint8(2)
memory usage: 4.1 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int32
dtypes: int32(1)
memory usage: 492.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 6 columns):
src_bytes    22543 non-null int64
dst_bytes    22543 non-null int64
hot          22543 non-null int64
count        22543 non-null int64
ecr_i        22543 non-null uint8
http         22543 non-null uint8
dtypes: int64(4), uint8(2)
memory usage: 748.6 KB
C info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543

C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [9]:
from keras.models import load_model
# Recreate the exact same model purely from the file

new_model = load_model('6featureModel.h5')
time1= datetime.datetime.now()
new_predictions = new_model.predict(X_test)
time2= datetime.datetime.now()
time=((time2-time1)/22543)
print(time)

0:00:00.000021



# Testing Time_ Model created with 4 features

In [10]:
alldata.head()
#col_list=['dst_host_diff_srv_rate','dst_host_serror_rate','serror_rate','dst_host_same_src_port_rate','S0','dst_host_srv_serror_rate','class']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot','dst_host_srv_serror_rate','is_guest_login','icmp','REJ','dst_host_serror_rate','dst_host_srv_diff_host_rate','tcp','urp_i','dst_host_rerror_rate','finger']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count']


#10 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','ecr_i','http','private']

#8 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_same_srv_rate','ecr_i','http','private']

#6 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','ecr_i','http']

#4 feature set
col_list=['class','src_bytes', 'dst_bytes','ecr_i','http']

#3 feature set
#col_list=['class','src_bytes', 'ecr_i','http']

#col_list=['class','ecr_i','http','src_bytes','hot','dst_bytes','dst_host_same_srv_rate']
alldata2=alldata[col_list]
traindata=alldata2.ix[0:trainlen-1,:]
testdata=alldata2.ix[trainlen:alldata2.shape[0],:]


#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))

X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 4 columns):
src_bytes    125973 non-null int64
dst_bytes    125973 non-null int64
ecr_i        125973 non-null uint8
http         125973 non-null uint8
dtypes: int64(2), uint8(2)
memory usage: 2.2 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int32
dtypes: int32(1)
memory usage: 492.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 4 columns):
src_bytes    22543 non-null int64
dst_bytes    22543 non-null int64
ecr_i        22543 non-null uint8
http         22543 non-null uint8
dtypes: int64(2), uint8(2)
memory usage: 396.3 KB
C info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 1 columns):
class    22543 non-null int32
dtypes: int32(1)
memory usage: 88.1 KB


C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [11]:
from keras.models import load_model
# Recreate the exact same model purely from the file

new_model = load_model('4featureModel.h5')
time1= datetime.datetime.now()
new_predictions = new_model.predict(X_test)
time2= datetime.datetime.now()
time=((time2-time1)/22543)
print(time)

0:00:00.000022



# Testing Time_ Model created with 3 features


In [12]:
alldata.head()
#col_list=['dst_host_diff_srv_rate','dst_host_serror_rate','serror_rate','dst_host_same_src_port_rate','S0','dst_host_srv_serror_rate','class']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot','dst_host_srv_serror_rate','is_guest_login','icmp','REJ','dst_host_serror_rate','dst_host_srv_diff_host_rate','tcp','urp_i','dst_host_rerror_rate','finger']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count','private','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','hot']
#col_list=['class','ecr_i','src_bytes','http','dst_bytes','count']


#10 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_same_src_port_rate','ecr_i','http','private']

#8 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','dst_host_same_srv_rate','ecr_i','http','private']

#6 feature set
#col_list=['class','src_bytes', 'dst_bytes','hot','count','ecr_i','http']

#4 feature set
#col_list=['class','src_bytes', 'dst_bytes','ecr_i','http']

#3 feature set
col_list=['class','src_bytes', 'ecr_i','http']

#col_list=['class','ecr_i','http','src_bytes','hot','dst_bytes','dst_host_same_srv_rate']
alldata2=alldata[col_list]
traindata=alldata2.ix[0:trainlen-1,:]
testdata=alldata2.ix[trainlen:alldata2.shape[0],:]


#X = traindata.iloc[:,1:42]
#Y = traindata.iloc[:,0]
#C = testdata.iloc[:,0]
#T = testdata.iloc[:,1:42]

X = traindata.drop('class', axis=1)
Y = traindata[['class']]
T = testdata.drop('class',axis=1)
C = testdata[['class']]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

print("X info")
X.info()
print("Y info")
Y.info()
print("T info")
T.info()
print("C info")
C.info()
y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))

X info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 3 columns):
src_bytes    125973 non-null int64
ecr_i        125973 non-null uint8
http         125973 non-null uint8
dtypes: int64(1), uint8(2)
memory usage: 1.2 MB
Y info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 1 columns):
class    125973 non-null int32
dtypes: int32(1)
memory usage: 492.2 KB
T info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 3 columns):
src_bytes    22543 non-null int64
ecr_i        22543 non-null uint8
http         22543 non-null uint8
dtypes: int64(1), uint8(2)
memory usage: 220.2 KB
C info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 125973 to 148515
Data columns (total 1 columns):
class    22543 non-null int32
dtypes: int32(1)
memory usage: 88.1 KB


C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Rashmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [13]:
from keras.models import load_model
# Recreate the exact same model purely from the file

new_model = load_model('3featureModel.h5')
time1= datetime.datetime.now()
new_predictions = new_model.predict(X_test)
time2= datetime.datetime.now()
time=((time2-time1)/22543)
print(time)

0:00:00.000025
